# Russian Synodal Bible (1885)

## Import

### Libraries

In [ ]:
#import os
#from lxml import etree
from backend import *

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [ ]:
bibleTXT = './texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './texts/bible/booksDict.json'
bibleJSON = './texts/bible/bible.json'
bibleIdJSON = './texts/bible/bibleID.json'
bibleXML = './texts/bible/bible.xml'
bibFrazyTXT = './texts/bible/Dubrovina_Slovar_Bibleyskikh_Frazeologizmov.txt'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [ ]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

### Tidy Up

### Find Parts

### Assign OHCO Labels

### Swap Full Book Name for Abbreviation

### Export to JSON

## Processing

### OHCO DFs

#### Testaments

#### Books

#### Chapters

### Make XML

## Token DFs

In [ ]:
TokenDf = pd.read_pickle('./proc/BibleVerseTokenDf.pkl')#.set_index(['p_id', 'token_id'])
TokenDf

In [ ]:
def GetRankDf(sourceDf, col='lemma', stop=False): 
    if stop: 
        sourceDf = TokenDf.loc[~TokenDf[tokenCols[9:]].isna().all(1)]
    RankDf = sourceDf[col].value_counts().to_frame().rename(columns={col:'n'})
    RankDf.index.name = col
    RankDf['rank'] = np.arange(1,len(RankDf)+1)
    return RankDf

In [ ]:
GetRankDf('lemma', stop=True)[:50].plot(x='rank')

In [ ]:
GetRankDf('pos')

In [ ]:
GetRankDf('rel')

In [ ]:
GetRankDf('anim')

In [ ]:
VocabDf.iloc[:50].plot(x='rank');

## Bible Phrase Dictionary

In [ ]:
with open(bibFrazyTXT) as bibleFrazy: 
    bibleFrazyLines = bibleFrazy.readlines()

In [ ]:
BibleFrazyDf = pd.DataFrame(bibleFrazyLines).rename(columns={0:'fraza'})
BibleFrazyDf.loc[:,'fraza'] = BibleFrazyDf.loc[:,'fraza'].apply(lambda x: x.strip().strip('\n'))
BibleFrazyDf = BibleFrazyDf.loc[BibleFrazyDf.fraza != '']
BibleFrazyDf.loc[112:30185]

In [ ]:
BibleFrazyDf.loc[30189:].fraza.to_list()301853018530185

In [ ]:
BibleFrazyDf.loc[34064].fraza

In [ ]:
BibleFrazyDf.loc[33944:].fraza.loc[BibleFrazyDf.fraza.str.match('^[а-я]', case=True)].to_frame()

In [ ]:
BibleFrazyDf.loc[33944:].fraza.loc[BibleFrazyDf.fraza.str.match('^\s')]

In [ ]:
FrazyAbbrsDf = BibleFrazyDf.iloc[:105].fraza.str.split(' – ', expand=True).rename(columns={0:"abbr", 1:"fraza"})
FrazyAbbrsDf.loc[:,'abbr'] = FrazyAbbrsDf.loc[:,'abbr'].apply(lambda x: x.strip())
FrazyAbbrsDf.loc[:,'fraza'] = FrazyAbbrsDf.loc[:,'fraza'].apply(lambda x: x.strip())
FrazyAbbrsDict = FrazyAbbrsDf.set_index('abbr').to_dict().get('fraza')